**Links**:

- [Текст с ошибками](https://yandex.ru/search/?text=Efficient%20Dependency%20Parsing%20In%20Case%20Of%20Ungrammatical%20Sentences&clid=2411726&lr=2)
- [Распознавание текста с ошибками](https://ru.stackoverflow.com/questions/859694/%D0%A0%D0%B0%D1%81%D0%BF%D0%BE%D0%B7%D0%BD%D0%B0%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5-%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0-%D1%81-%D0%BE%D1%88%D0%B8%D0%B1%D0%BA%D0%B0%D0%BC%D0%B8)
- [Опечатки с учетом контекста](https://habr.com/ru/post/346618/)
- [Spell-Corrector](http://norvig.com/spell-correct.html)
- [Project JetBrains](https://internship.jetbrains.com/projects/848/)
- [Supar](https://github.com/yzhangcs/parser)
- [Error-repair Dependency](https://www.aclweb.org/anthology/P17-2030.pdf)
- [Parser](https://github.com/HHashemi?tab=repositories)
- [Dependency Parser](https://medium.com/data-science-in-your-pocket/dependency-parsing-associated-algorithms-in-nlp-96d65dd95d3e)
- [Dependency NLP Parser](http://nlpprogress.com/english/dependency_parsing.html)
- [Error-repair Dependency Paper](https://www.researchgate.net/publication/318741850_Error-repair_Dependency_Parsing_for_Ungrammatical_Texts)

In [8]:
import numpy as np
import pandas as pd
import operator
import os

# Corus - NLP datasets
import corus
from corus import load_lenta

#NLTK - Natural Language Tool Kit
import nltk

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import words
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
nltk.download('punkt')

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import bigrams
from nltk import ngrams

#Other
from collections import Counter
import re
import string
from tqdm import notebook

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aptmess/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/aptmess/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /home/aptmess/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/aptmess/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
def download_file(file_name: str):
    os.system(f'wget -P ~/IHaskell/word_data/ {file_name}')

from corus.sources.meta import METAS
from corus.readme import format_metas, show_html, patch_readme

html = format_metas(METAS)
show_html(html)

In [11]:
!wget -P ~/IHaskell/word_data/ https://github.com/dialogue-evaluation/GramEval2020/archive/master.zip

--2021-04-09 19:54:07--  https://github.com/dialogue-evaluation/GramEval2020/archive/master.zip
Resolving github.com (github.com)... 140.82.121.4, 198.51.44.8, 198.51.45.8, ...
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/dialogue-evaluation/GramEval2020/zip/master [following]
--2021-04-09 19:54:07--  https://codeload.github.com/dialogue-evaluation/GramEval2020/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.121.9, 198.51.44.8, 198.51.45.8, ...
Connecting to codeload.github.com (codeload.github.com)|140.82.121.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/home/aptmess/IHaskell/word_data/master.zip.2’

master.zip.2            [          <=>       ]  24.58M  5.08MB/s    in 7.5s    

2021-04-09 19:54:15 (3.27 MB/s) - ‘/home/aptmess/IHaskell/word_data/master.zip.2’ saved [25777087]


In [2]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

FileNotFoundError: [Errno 2] No such file or directory: 'big.txt'